In [ ]:
import json
import pandas as pd
from urllib.request import urlopen



In [ ]:



url = "https://mosaicweb.com.br/aulas/python/vendas.json"
# salvar a resposta da url
response = urlopen(url)
data_json = json.loads(response.read())
df = pd.DataFrame.from_dict(data_json)







In [ ]:
df['Data da Compra']

0       01/01/2020
1       01/01/2020
2       01/01/2020
3       01/01/2020
4       01/01/2020
           ...    
9430    31/12/2022
9431    31/12/2022
9432    31/12/2022
9433    31/12/2022
9434    31/12/2022
Name: Data da Compra, Length: 9435, dtype: object

In [ ]:
df['Data da Compra'] = pd.to_datetime(df['Data da Compra'], format='%d/%m/%Y')

In [ ]:
df['Preço'].sum()


5871490.789999999

In [ ]:
def format_number(value, prefix = ''):
  for unit in ['', 'mil']:
    if value < 1000:
      return f'{prefix} {value:.2f} {unit}'
    value /= 1000
  return f'{prefix} {value:.2f} milhões'

In [ ]:
format_number(df['Preço'].sum(), 'R$')

'R$ 5.87 milhões'

In [ ]:
format_number(df.shape[0])

' 9.44 mil'

In [ ]:
df_rec_estado = df.groupby('Local da compra')[['Preço']].sum()


In [ ]:
df_rec_estado = df.groupby('Local da compra')[['Preço']].sum()

In [ ]:
df_rec_estado = df.drop_duplicates(subset='Local da compra')[['Local da compra', 'lat', 'lon']]


In [ ]:
df_rec_estado = df.groupby('Local da compra')[['Preço']].sum()
df_rec_estado = df.drop_duplicates(subset='Local da compra')[['Local da compra', 'lat','lon']].merge(df_rec_estado, left_on='Local da compra', right_index=True).sort_values('Preço', ascending=False)

In [ ]:
df_rec_mensal = df.set_index('Data da Compra').groupby(pd.Grouper(freq='M'))['Preço'].sum().reset_index()

In [ ]:
df_rec_mensal['Mes'] = df_rec_mensal['Data da Compra'].dt.month_name()
df_rec_mensal['Ano'] = df_rec_mensal['Data da Compra'].dt.year

df_rec_mensal

,Data da Compra,Preço,Mes,Ano
0,2020-01-31,143060.68,January,2020
1,2020-02-29,143867.81,February,2020
2,2020-03-31,160848.48,March,2020
3,2020-04-30,181531.80,April,2020
4,2020-05-31,162879.36,May,2020
5,2020-06-30,128893.49,June,2020
6,2020-07-31,158880.06,July,2020
7,2020-08-31,148909.59,August,2020
8,2020-09-30,154987.12,September,2020
9,2020-10-31,120876.93,October,2020


In [ ]:
df_rec_categoria = df.groupby('Categoria do Produto')[['Preço']].sum().sort_values('Preço',ascending=False)

In [ ]:
df_vendedores = pd.DataFrame(df.groupby('Vendedor')['Preço'].agg(['sum', 'count']))


In [ ]:
import plotly.express as px
grafico_map_estado = px.scatter_geo(
df_rec_estado, #estou passando o dataframe com meus dados
lat = 'lat', #coluna latitude
lon = 'lon', #coluna longitude
scope = 'south america', #estou restringindo meu mapa a américa do sul
size = 'Preço',
template = 'seaborn',
hover_name = 'Local da compra', #quando passar o nome em cima mostra o local
hover_data = {'lat': False, 'lon': False},
title = 'Receita por Estado'
)
grafico_map_estado

In [ ]:
grafico_rec_mensal = px.line(
df_rec_mensal,
x = 'Mes',
y = 'Preço',
markers = True,
range_y = (0, df_rec_mensal.max()),
color = 'Ano',
line_dash = 'Ano',
title = 'Receita Mensal'
)
grafico_rec_mensal.update_layout(yaxis_title = 'Receita')

In [ ]:
df_avalia = df.groupby('Local da compra')[['Avaliação da compra']].sum().sort_values('Avaliação da compra',ascending=False)

In [ ]:
grafico_avalia = px.bar(
df_avalia,
text_auto = True,
title = 'Quantidade de avaliações por estado'
)
grafico_avalia

In [ ]:
grafico_rec_estado = px.bar(
df_rec_estado.head(7),
x = 'Local da compra',
y = 'Preço',
text_auto = True,
title = 'Top Receita por Estados'
)
grafico_rec_estado

In [ ]:
grafico_rec_categoria = px.bar(
df_rec_categoria.head(7),
text_auto = True,
title = 'Top 7 Categorias com Maior Receita'
)
grafico_rec_categoria


In [ ]:
grafico_rec_vendedores = px.bar(
df_vendedores[['sum']].sort_values('sum', ascending=False).head(7),
x = 'sum',
y = df_vendedores[['sum']].sort_values('sum', ascending=False).head(7).index,
text_auto = True,
title = 'Top 7 Vendedores por Receita'
)
grafico_rec_vendedores

In [ ]:
grafico_vendas_vendedores = px.bar(
df_vendedores[['count']].sort_values('count', ascending=False).head(7),
x = 'count',
y = df_vendedores[['count']].sort_values('count', ascending=False).head(7).index,
text_auto = True,
title = 'Top 7 Vendedores por Venda'
)
grafico_vendas_vendedores

In [ ]:
!pip install -q streamlit


In [ ]:
!npm install localtunnel


In [ ]:
import urllib
print("SENHA/Enpoint IP para o túnel local:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

SENHA/Enpoint IP para o túnel local: 34.32.195.95


In [ ]:
%%writefile app.py
import streamlit as st
from aula_5 import *
st.set_page_config(layout='wide')
st.title("Dashboard de Vendas :shopping_trolley:")
st.sidebar.title('Filtro de Vendedores')
filtro_vendedor = st.sidebar.multiselect(
'Vendedores',
df['Vendedor'].unique(),
)

if filtro_vendedor:
  df = df[df['Vendedor'].isin(filtro_vendedor)]

aba1, aba2, aba3, aba4 = st.tabs(['Dataset', 'Receita', 'Vendedores','Avaliações'])
with aba1:
  st.dataframe(df)
with aba2:
  coluna1, coluna2 = st.columns(2)
  with coluna1:
    st.metric('Receita Total', format_number(df['Preço'].sum(), 'R$'))
    st.plotly_chart(grafico_map_estado, use_container_width=True)
    st.plotly_chart(grafico_rec_estado, use_container_width=True)
  with coluna2:
    st.metric('Quantidade de Vendas', format_number(df.shape[0]))
    st.plotly_chart(grafico_rec_mensal, use_container_width=True)
    st.plotly_chart(grafico_rec_categoria, use_container_width=True)
with aba3:
  coluna1, coluna2 = st.columns(2)
  with coluna1:
    st.plotly_chart(grafico_rec_vendedores, use_container_width=True)
  with coluna2:
    st.plotly_chart(grafico_vendas_vendedores, use_container_width=True)
with aba4:
    st.plotly_chart(grafico_avalia, use_container_width=True)


Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501


npx: installed 22 in 2.068s
your url is: https://wide-towns-think.loca.lt
